Пример задачи (https://www.kaggle.com/c/titanic/data) про спасенных с "Титаника"

Описание полей:
- PassengerId — идентификатор пассажира
- Survival — поле в котором указано спасся человек (1) или нет (0)
- Pclass — содержит социально-экономический статус:
    - высокий
    - средний
    - низкий
- Name — имя пассажира
- Sex — пол пассажира
- Age — возраст
- SibSp — содержит информацию о количестве родственников 2-го порядка (муж, жена, братья, сетры)
- Parch — содержит информацию о количестве родственников на борту 1-го порядка (мать, отец, дети)
- Ticket — номер билета
- Fare — цена билета
- Cabin — каюта
- Embarked — порт посадки
    - C — Cherbourg
    - Q — Queenstown
    - S — Southampton

Суть задачи состоит в том, чтобы с помощью методов машинного обучения построить модель, которая прогнозировала бы спасется человек или нет.

In [1]:
import pandas as pd
import numpy as np
import csv as csv
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_df = pd.read_csv('Data/train.csv', header=0)        # Загрузка файла с обучающими данными в dataframe
train_df[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [3]:
# Для классификации необходимо конвертировать все строковые значения в целочисленные
# female = 0, Male = 1
train_df['Gender'] = train_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
train_df[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,1


In [ ]:
# Есть несколько записей, которые не содержат значения в поле Embarked. Ниже эти пробелы заполняются.
# Вникать в этот процесс не является необходимым
if len(train_df.Embarked[ train_df.Embarked.isnull() ]) > 0:
    train_df.Embarked[ train_df.Embarked.isnull() ] = train_df.Embarked.dropna().mode().values

Ports = list(enumerate(np.unique(train_df['Embarked'])))    # получение списка портов посадки,
Ports_dict = { name : i for i, name in Ports }              # составление словаря  Ports : index
train_df.Embarked = train_df.Embarked.map( lambda x: Ports_dict[x]).astype(int)     # Заменяем строковые значения на числовые

# Для записей, которые не содержат значения в поле Age, установить значение как медиану от всех возрастов
median_age = train_df['Age'].dropna().median()
if len(train_df.Age[ train_df.Age.isnull() ]) > 0:
    train_df.loc[ (train_df.Age.isnull()), 'Age'] = median_age

# Удаляем колонки:  Name, Cabin, Ticket, и Sex
train_df = train_df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'PassengerId'], axis=1) 

# Получение numpy array, который будет использоваться при классификации
train_data = train_df.values

In [5]:
# Таким образом имеем dataset такого вида
train_df[:10]

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,Gender
0,0,3,22,1,0,7.2500,2,1
1,1,1,38,1,0,71.2833,0,0
2,1,3,26,0,0,7.9250,2,0
3,1,1,35,1,0,53.1000,2,0
4,0,3,35,0,0,8.0500,2,1
5,0,3,28,0,0,8.4583,1,1
6,0,1,54,0,0,51.8625,2,1
7,0,3,2,3,1,21.0750,2,1
8,1,3,27,0,2,11.1333,2,0
9,1,2,14,1,0,30.0708,0,0


In [6]:
# Для данной задачи имеется файл test.csv, который содержит записи для тестирования классификации
# Однако пойдем другим путем, и получим обучающий и тестовые наборы, разбив train_data на части
train_part = 0.5
test_part = 1 - train_part
count = len(train_data)

tr_data = train_data[0:int(count*train_part)]
tst_data = train_data[int(count*train_part + 1)::]
print(len(tr_data), len(tst_data))

445 445


In [8]:
# Осуществляем обучение классификатора
# tr_data[0::,1::] - передача значений X: Pclass, Age, SibSp, Parch, Fare, Embarked, Gender
# tr_data[0::,0] - передача значений Y: Survived
forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit( tr_data[0::,1::], tr_data[0::,0] )

# Получаем среднюю точность классификации на тестовых данных
print( forest.score(tst_data[0::,1::], tst_data[0::,0] ) )

0.788764044944
